# Предсказание стоимости жилья

В проекте вам нужно обучить модель линейной регрессии на данных о жилье в Калифорнии в 1990 году. На основе данных нужно предсказать медианную стоимость дома в жилом массиве. Обучите модель и сделайте предсказания на тестовой выборке. Для оценки качества модели используйте метрики RMSE, MAE и R2.

## Используемые библиотеки

In [1]:
import pandas as pd
import numpy as np

import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.types import *
import pyspark.sql.functions as F

from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler, Imputer
from pyspark.ml.regression import LinearRegression
from pyspark.ml.feature import OneHotEncoder

from pyspark.ml.evaluation import RegressionEvaluator

    
RANDOM_SEED = 12345

In [2]:
# Инициализируем Spark сессию

spark = SparkSession.builder.master('local').appName('Housing price prediction - Linear regression').getOrCreate()

# Загрузим наш датасет
data = spark.read.load('/datasets/housing.csv', format='csv', sep=',', inferSchema=True, header=True)

## Осмотр данных

In [3]:
data.printSchema()
data.show()
# Выведем названия и тип колонок для более удобного осмотра
print(pd.DataFrame(data.dtypes, columns=['column', 'type']).head(10))

root
 |-- longitude: double (nullable = true)
 |-- latitude: double (nullable = true)
 |-- housing_median_age: double (nullable = true)
 |-- total_rooms: double (nullable = true)
 |-- total_bedrooms: double (nullable = true)
 |-- population: double (nullable = true)
 |-- households: double (nullable = true)
 |-- median_income: double (nullable = true)
 |-- median_house_value: double (nullable = true)
 |-- ocean_proximity: string (nullable = true)

+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|longitude|latitude|housing_median_age|total_rooms|total_bedrooms|population|households|median_income|median_house_value|ocean_proximity|
+---------+--------+------------------+-----------+--------------+----------+----------+-------------+------------------+---------------+
|  -122.23|   37.88|              41.0|      880.0|         129.0|     322.0|     126.0|       8.3252|          452600.0|       NEAR B

In [4]:
# Посмотрим на базовые статистики

data.toPandas().describe()

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value
count,20640.000000,20640.000000,20640.000000,20640.000000,20433.000000,20640.000000,20640.000000,20640.000000,20640.000000
mean,-119.569704,35.631861,28.639486,2635.763081,537.870553,1425.476744,499.539680,3.870671,206855.816909
std,2.003532,2.135952,12.585558,2181.615252,421.385070,1132.462122,382.329753,1.899822,115395.615874
min,-124.350000,32.540000,1.000000,2.000000,1.000000,3.000000,1.000000,0.499900,14999.000000
25%,-121.800000,33.930000,18.000000,1447.750000,296.000000,787.000000,280.000000,2.563400,119600.000000
50%,-118.490000,34.260000,29.000000,2127.000000,435.000000,1166.000000,409.000000,3.534800,179700.000000
75%,-118.010000,37.710000,37.000000,3148.000000,647.000000,1725.000000,605.000000,4.743250,264725.000000
max,-114.310000,41.950000,52.000000,39320.000000,6445.000000,35682.000000,6082.000000,15.000100,500001.000000


**Вывод**

- 10 колонок и 20 640 строк в датасете
- Видно, что имееются пропуски в колонке `total_bedrooms`
- В некоторых колонках очень большой разброс между минимальным и максимальным значениями
- 1 колонка с категориальными признаками и 9 с числовыми признаками

# Подготовка данных

In [5]:
# Проверим и посчитаем кол-во пропусков в данных
def missing_values(data):
    columns = data.columns

    for column in columns:
        if column != 'ocean_proximity':
            check_col = F.col(column).cast(FloatType())
            print(column, data.filter(F.isnull(check_col) == True).count())
            
missing_values(data)

longitude 0
latitude 0
housing_median_age 0
total_rooms 0
total_bedrooms 207
population 0
households 0
median_income 0
median_house_value 0


207 пропусков - заполним их медианным значением колонки. Тк это ~1% от всего датасета - такой способ заполнения пропусков не должен ухудшить точность предсказания. 

Можно, конечно, создать модель и предсказать значения в пропусках

In [6]:
imputer = Imputer(inputCols=['total_bedrooms'], outputCols=['total_bedrooms']).setStrategy('median')
data = imputer.fit(data).transform(data)

missing_values(data)
data.describe().toPandas()

longitude 0
latitude 0
housing_median_age 0
total_rooms 0
total_bedrooms 0
population 0
households 0
median_income 0
median_house_value 0


,summary,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,median_house_value,ocean_proximity
0,count,20640,20640,20640,20640,20640,20640,20640,20640,20640,20640
1,mean,-119.56970445736148,35.6318614341087,28.639486434108527,2635.7630813953488,536.8388565891473,1425.4767441860465,499.5396802325581,3.8706710029070246,206855.81690891474,None
2,stddev,2.003531723502584,2.135952397457101,12.58555761211163,2181.6152515827944,419.39187792168866,1132.46212176534,382.3297528316098,1.899821717945263,115395.61587441359,None
3,min,-124.35,32.54,1.0,2.0,1.0,3.0,1.0,0.4999,14999.0,<1H OCEAN
4,max,-114.31,41.95,52.0,39320.0,6445.0,35682.0,6082.0,15.0001,500001.0,NEAR OCEAN


### Разделим колонки для последующей трансформации

In [7]:
categorical_cols = 'ocean_proximity'
numerical_cols = ['longitude', 
                   'latitude', 
                   'housing_median_age', 
                   'total_rooms', 
                   'total_bedrooms', 
                   'population', 
                   'households', 
                   'median_income']
target = 'median_house_value'

### Трансформация категориальных признаков

In [8]:
indexer = StringIndexer(inputCol=categorical_cols, outputCol=categorical_cols + '_idx')

data = indexer.fit(data).transform(data)

cols = [c for c in data.columns if (c.startswith(categorical_cols))]
data.select(cols).show(3) 

+---------------+-------------------+
|ocean_proximity|ocean_proximity_idx|
+---------------+-------------------+
|       NEAR BAY|                3.0|
|       NEAR BAY|                3.0|
|       NEAR BAY|                3.0|
+---------------+-------------------+
only showing top 3 rows



#### Дополнительно создадим OHE-кодирование для категориального признака

In [9]:
encoder = OneHotEncoder(inputCol=categorical_cols + '_idx', outputCol=categorical_cols + '_ohe')

data = encoder.transform(data)

cols = [c for c in data.columns if (c.startswith(categorical_cols))]
data.select(cols).show(3) 

+---------------+-------------------+-------------------+
|ocean_proximity|ocean_proximity_idx|ocean_proximity_ohe|
+---------------+-------------------+-------------------+
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
|       NEAR BAY|                3.0|      (4,[3],[1.0])|
+---------------+-------------------+-------------------+
only showing top 3 rows



In [10]:
categorical_assembler = VectorAssembler(inputCols=[categorical_cols + '_ohe'], outputCol = 'categorical_features')

data = categorical_assembler.transform(data)


### Трансформация числовых признаков

In [11]:
numerical_assembler = VectorAssembler(inputCols=numerical_cols, outputCol='numerical_features')

data = numerical_assembler.transform(data)

In [12]:
# Применим скалирование, чтобы сильные выбросы не смещали предсказание модели

standardScaler = StandardScaler(inputCol='numerical_features', outputCol='numerical_features_scaled')
data = standardScaler.fit(data).transform(data)

In [13]:
# Посмотрим на получившиеся колонки

for i in data.columns:
    print(i)

longitude
latitude
housing_median_age
total_rooms
total_bedrooms
population
households
median_income
median_house_value
ocean_proximity
ocean_proximity_idx
ocean_proximity_ohe
categorical_features
numerical_features
numerical_features_scaled


In [14]:
# Соберем категориальные и числовые признаки

all_features = ['categorical_features', 'numerical_features_scaled']

final_assembler = VectorAssembler(inputCols=all_features, 
                                 outputCol='features')

data = final_assembler.transform(data)

data.select(all_features).show(3)

+--------------------+-------------------------+
|categorical_features|numerical_features_scaled|
+--------------------+-------------------------+
|       (4,[3],[1.0])|     [-61.007269596069...|
|       (4,[3],[1.0])|     [-61.002278409814...|
|       (4,[3],[1.0])|     [-61.012260782324...|
+--------------------+-------------------------+
only showing top 3 rows



# Обучение моделей

In [15]:
# Разделим датасет

train_data, test_data = data.randomSplit([.8, .2], seed=RANDOM_SEED)

print(train_data.count(), test_data.count())

16463 4177


## Обучение модели на всех данных

In [16]:
lr_full = LinearRegression(labelCol=target, featuresCol='features')

model_full = lr_full.fit(train_data)

In [17]:
predictions_full = model_full.transform(test_data)

predictedLabels_full = predictions_full.select(target, 'prediction')

predictedLabels_full.show()

+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|          111400.0|190296.03048393782|
|           50800.0| 212626.7991843545|
|           73200.0|123821.45584555529|
|           70000.0|147366.33497304423|
|           74600.0|102818.04068596382|
|          107000.0| 188681.2917989632|
|           85100.0| 177896.3006086601|
|           90600.0| 178738.3447544491|
|           92700.0|182998.17272855528|
|          100600.0| 190340.0296095647|
|           67500.0|146309.03733594855|
|           99600.0| 184647.2714346042|
|           94800.0| 215796.0043739425|
|           86900.0|150594.16947201127|
|           92800.0|  206524.572040305|
|           96100.0| 154126.6468826388|
|           90200.0| 151983.3717899355|
|           92600.0|167682.66757511767|
|           75000.0|113516.35807134723|
|          150000.0| 119872.0266682012|
+------------------+------------------+
only showing top 20 rows



In [18]:
# RMSE MAE R2

RMSE = RegressionEvaluator(labelCol=target).evaluate(predictions_full)
MAE = RegressionEvaluator(labelCol=target, metricName='mae').evaluate(predictions_full)
R2 = RegressionEvaluator(labelCol=target, metricName='r2').evaluate(predictions_full)

print('RMSE: ', RMSE)
print('MAE: ', MAE)
print('R2: ', R2)

RMSE:  69035.31031569629
MAE:  49651.64429302908
R2:  0.6586737670520186


## Обучение модели на числовых фичах

In [19]:
# Обучим модель без категориальных фичей, использовав ранее определенные числовые колонки
lr_cut = LinearRegression(labelCol=target, featuresCol='numerical_features_scaled')

model_cut = lr_cut.fit(train_data)

In [20]:
predictions_cut = model_cut.transform(test_data)

predictedLabels_cut = predictions_cut.select(target, 'prediction')

predictedLabels_cut.show()

+------------------+------------------+
|median_house_value|        prediction|
+------------------+------------------+
|          111400.0|164235.37262760988|
|           50800.0| 183287.0400344138|
|           73200.0| 73900.74679343449|
|           70000.0|115666.88016890315|
|           74600.0|  52403.1854412267|
|          107000.0|159936.30914820544|
|           85100.0| 149382.4614269901|
|           90600.0|147232.77891856432|
|           92700.0|152115.75639728317|
|          100600.0|157693.86561529106|
|           67500.0| 114116.1406116467|
|           99600.0| 152148.8145630043|
|           94800.0|195689.13781347545|
|           86900.0|117434.13975783857|
|           92800.0| 174140.4559944435|
|           96100.0| 119744.9209678336|
|           90200.0| 115014.1973974742|
|           92600.0| 131352.9400998312|
|           75000.0| 77943.93870104104|
|          150000.0| 84616.14367153728|
+------------------+------------------+
only showing top 20 rows



In [21]:
# RMSE MAE R2

RMSE_cut = RegressionEvaluator(labelCol=target).evaluate(predictions_cut)
MAE_cut = RegressionEvaluator(labelCol=target, metricName='mae').evaluate(predictions_cut)
R2_cut = RegressionEvaluator(labelCol=target, metricName='r2').evaluate(predictions_cut)

print('RMSE: ', RMSE_cut)
print('MAE: ', MAE_cut)
print('R2: ', R2_cut)

RMSE:  70077.29458745479
MAE:  50991.78979509102
R2:  0.6482923951034327


# Анализ результатов

In [22]:
# Соберем наши метрики в общую таблицу для наглядного сравнения

pred_data = [
    ['full data predictions', 
    round(RMSE, 3),
    round(MAE, 3),
    round(R2, 3)], 
    ['num data predictions', 
    round(RMSE_cut, 3),
    round(MAE_cut, 3),
    round(R2_cut, 3)]]

pred_cols = ['data_name', 
            'RMSE', 
            'MAE', 
            'R2']

pred_data_models = pd.DataFrame(data=pred_data, columns=pred_cols)

pred_data_models

,data_name,RMSE,MAE,R2
0,full data predictions,69035.310,49651.644,0.659
1,num data predictions,70077.295,50991.790,0.648


**Вывод**

- Были обучены два варианта моделей:
    - С полным набором фичей
    - Только с численными фичами
- Модель, обученная на полном наборе данных показала более точный результат:
    - Значение R2 выше
    - Значения RMSE и MAE ниже
